# GITHUB DATA SCRAPING

In [1]:
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
pd.options.display.max_rows = 9999

In [22]:
def get_user_details(username, token):
    url = f'https://api.github.com/users/{username}'
    headers = {'Authorization': f'token {token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error fetching details for {username}: {response.status_code}')
        return None

def search_github_users(query, token, max_results=1000):
    url = 'https://api.github.com/search/users'
    headers = {'Authorization': f'token {token}'}
    users = []
    page = 1
    per_page = 100
    while True:
        params = {
            'q': query,
            'per_page': per_page,
            'page': page,
            'sort': 'repositories',
            'order': 'desc'
        
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(response.status_code)
            break
        data = response.json()
        global total_count
        total_count=data["total_count"]
        users.extend(data['items'])
        if len(data['items']) < per_page or len(users) >= max_results:
            break
        page += 1
    return users



In [26]:
def create_Talent_Table(query):
    token = 'Enter Your Token'
    users = search_github_users(query, token)
    Username=[]
    URL=[]
    Company=[]
    Email=[]
    HireAble=[]
    Public_Repos=[]
    Public_Gists=[]
    for user in users:
        username = user['login']
        user_details = get_user_details(username, token)
        if user_details:
            Username.append(username)
            URL.append(user_details['html_url'])
            Company.append(user_details.get('company','Not Available'))
            Email.append(user_details.get('email','Not available'))
            HireAble.append(user_details.get('hireable'))
            Public_Repos.append(user_details.get('public_repos'))
            Public_Gists.append(user_details.get('public_gists'))
            
    df = pd.DataFrame({'Username': Username,'Github Link': URL,'Company': Company,'Email':Email,'HireAble' : HireAble,'Public_Repos':Public_Repos,'Public_Gists':Public_Gists}) 
    return df

In [45]:
query1 = 'language:Python location:Bangalore repos:<9 created:<2019-01-01 followers:<1 type:user'
query2 = 'language:Python location:Bangalore repos:<9 created:<2019-01-01 followers:>=1 type:user'
query3 = 'language:Python location:Bangalore repos:<9 created:>2019-01-01 followers:<1 type:user'
query4 = 'language:Python location:Bangalore repos:<9 created:>2019-01-01 followers:>=1 type:user'
query5 = 'language:Python location:Bangalore repos:>=9 created:<2018-01-01 followers:<=1 type:user'
query6 = 'language:Python location:Bangalore repos:>=9 created:<2018-01-01 followers:>1 type:user'
query7 = 'language:Python location:Bangalore repos:>=9 created:>2018-01-01 followers:<=1 type:user'
query8 = 'language:Python location:Bangalore repos:>=9 created:>2018-01-01 followers:>1 type:user'
df_blr1=create_Talent_Table(query1)
df_blr2=create_Talent_Table(query2)
df_blr3=create_Talent_Table(query3)
df_blr4=create_Talent_Table(query4)
df_blr5=create_Talent_Table(query5)
df_blr6=create_Talent_Table(query6)
df_blr7=create_Talent_Table(query7)
df_blr8=create_Talent_Table(query8)

total_blr=total_count
df_hyd=create_Talent_Table('Hyderabad')
total_hyd=total_count

In [60]:
query1 = 'language:Python location:Hyderabad repos:<7 created:<2020-01-01 followers:<1 type:user'
query2 = 'language:Python location:Hyderabad repos:<7 created:<2020-01-01 followers:>=1 type:user'
query3 = 'language:Python location:Hyderabad repos:<7 created:>2020-01-01 followers:<1 type:user'
query4 = 'language:Python location:Hyderabad repos:<7 created:>2020-01-01 followers:>=1 type:user'
query5 = 'language:Python location:Hyderabad repos:>=7 created:<2019-01-01 followers:<=1 type:user'
query6 = 'language:Python location:Hyderabad repos:>=7 created:<2019-01-01 followers:>1 type:user'
query7 = 'language:Python location:Hyderabad repos:>=7 created:>2019-01-01 followers:<=1 type:user'
query8 = 'language:Python location:Hyderabad repos:>=7 created:>2019-01-01 followers:>1 type:user'
df_hyd1=create_Talent_Table(query1)
df_hyd2=create_Talent_Table(query2)
df_hyd3=create_Talent_Table(query3)
df_hyd4=create_Talent_Table(query4)
df_hyd5=create_Talent_Table(query5)
df_hyd6=create_Talent_Table(query6)
df_hyd7=create_Talent_Table(query7)
df_hyd8=create_Talent_Table(query8)

In [72]:
df_blr = pd.concat([df_blr1, df_blr2, df_blr3, df_blr4, df_blr5, df_blr6, df_blr7, df_blr8], axis=0, ignore_index=True)
df_blr = df_blr.drop_duplicates()
df_blr = df_blr.sort_values(by='Public_Repos', ascending=False)
df_blr.reset_index(drop=True, inplace=True)

In [73]:
df_hyd = pd.concat([df_hyd1, df_hyd2, df_hyd3, df_hyd4, df_hyd5, df_hyd6, df_hyd7, df_hyd8], axis=0, ignore_index=True)
df_hyd = df_hyd.drop_duplicates()
df_hyd = df_hyd.sort_values(by='Public_Repos', ascending=False)
df_hyd.reset_index(drop=True, inplace=True)

In [76]:
def scrape_Github_Bangalore_To_Excel(writer,df)  :
    df.to_excel(writer,sheet_name='Bangalore Python Developers')

def scrape_Github_Hyderabad_To_Excel(writer,df)  :
    df.to_excel(writer,sheet_name='Hyderabad Python Developers')
    
with pd.ExcelWriter('Profile_details.xlsx') as writer:
    scrape_Github_Bangalore_To_Excel(writer,df_blr)
    scrape_Github_Hyderabad_To_Excel(writer,df_hyd)